In [1]:
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
repository = pd.read_csv("../autos.csv", encoding='ISO-8859-1')

In [3]:
repository = repository.drop('dateCrawled', axis=1)\
                       .drop('dateCreated', axis=1)\
                       .drop('nrOfPictures', axis=1)\
                       .drop('postalCode', axis=1)\
                       .drop('name', axis=1)\
                       .drop('seller', axis=1)\
                       .drop('offerType', axis=1)\
                       .drop('lastSeen', axis=1)

In [4]:
i1 = repository.loc[repository.price <= 10]
repository = repository[repository.price > 10]
i1 = repository.loc[repository.price >= 350000]
repository = repository[repository.price < 350000]
values = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

repository = repository.fillna(value=values)


In [5]:
previsors = repository.iloc[:, 1:13].values
real_price = repository.iloc[:, 0].values

In [6]:

labelEncoder_prev = LabelEncoder()
previsors[:,0] = labelEncoder_prev.fit_transform(previsors[:,0])
previsors[:,1] = labelEncoder_prev.fit_transform(previsors[:,1])
previsors[:,3] = labelEncoder_prev.fit_transform(previsors[:,3])
previsors[:,5] = labelEncoder_prev.fit_transform(previsors[:,5])
previsors[:,8] = labelEncoder_prev.fit_transform(previsors[:,8])
previsors[:,9] = labelEncoder_prev.fit_transform(previsors[:,9])
previsors[:,10] = labelEncoder_prev.fit_transform(previsors[:,10])

In [7]:
encoder = OneHotEncoder(sparse=False)
cols_to_encode = [0, 1, 3, 5, 8, 9, 10]
columns_to_encode = previsors[:, cols_to_encode]
encoded_columns = encoder.fit_transform(columns_to_encode)
dataset_encoded = np.delete(previsors, cols_to_encode, axis=1)
dataset_encoded = np.concatenate((dataset_encoded, encoded_columns), axis=1)
previsorsEncoded = dataset_encoded.astype(float)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
def createNetwork(loss):
    regressor = Sequential()
    regressor.add(Dense(units=158, activation='relu', input_dim=316))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))
    regressor.compile(loss=loss, optimizer='adam', metrics=['mean_absolute_error'])

    return regressor

In [9]:
classifier = KerasRegressor(build_fn=createNetwork)
parameters = {
    'batch_size': [10],
    'epochs': [100],
    'loss': ['mean_squared_error' , 'mean_absolute_error' , 'mean_absolute_percentage_error' , 'mean_squared_logarithmic_error', 'squared_hinge']
}

C:\Users\Greze\AppData\Local\Temp\ipykernel_16820\2290633016.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasRegressor(build_fn=createNetwork)


In [10]:
gridSearch = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=3,
    n_jobs=5
)

gridSearch = gridSearch.fit(
    previsorsEncoded,
    real_price,
    verbose=0,
    use_multiprocessing = True,
)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


In [12]:
gridSearch.best_params_

{'batch_size': 10, 'epochs': 100, 'loss': 'mean_squared_error'}